In [1]:
#!pip3 install rdflib
#!pip3 install rdfpandas
#!pip3 install rdflib-jsonld
#!pip3 install SPARQLWrapper

In [1]:
import rdflib
import pandas as pd
import json
from collections import Counter
from rdflib import Graph, plugin
from rdflib.serializer import Serializer
import sddk

import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from google.oauth2 import service_account # based on google-auth library

In [10]:
pd.options.display.max_columns = 999

In [10]:
# download 
#!wget -O ../data/nomisma.rdf http://numismatics.org/ocre/nomisma.rdf

--2021-03-19 10:46:50--  http://numismatics.org/ocre/nomisma.rdf
Resolving numismatics.org (numismatics.org)... 161.47.117.67
Connecting to numismatics.org (numismatics.org)|161.47.117.67|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/rdf+xml]
Saving to: ‘../data/nomisma.rdf’

../data/nomisma.rdf     [         <=>        ] 118.48M  11.4MB/s    in 11s     

2021-03-19 10:48:25 (10.3 MB/s) - ‘../data/nomisma.rdf’ saved [124236780]



In [5]:
%%time
g = rdflib.Graph()
g.parse('../data/nomisma.rdf')

CPU times: user 1min 41s, sys: 252 ms, total: 1min 41s
Wall time: 1min 41s


<Graph identifier=N4a4340e51d434796879a70c019bef84f (<class 'rdflib.graph.Graph'>)>

In [3]:
%%time
# other option is to read the file directly from the source
#g = rdflib.Graph()
#g.parse("http://numismatics.org/ocre/nomisma.rdf")

<Graph identifier=N36edf143f03b4d94ae95412cedae417e (<class 'rdflib.graph.Graph'>)>

In [6]:
len(g)

1288846

In [14]:
nomisma_json_bytes = g.serialize(format='json-ld', indent=4)

In [15]:
type(nomisma_json_bytes)

bytes

In [18]:
# get the actual json
nomisma_json = literal_eval(nomisma_json_bytes.decode('utf8'))

In [25]:
with open('../data/nomisma.json', 'w') as f:
    json.dump(nomisma_json, f)

In [26]:
nomisma_json[0]

{'@id': 'http://numismatics.org/ocre/id/ric.6.sis.221',
 '@type': ['http://www.w3.org/2004/02/skos/core#Concept',
  'http://nomisma.org/ontology#TypeSeriesItem'],
 'http://nomisma.org/ontology#hasAuthority': [{'@id': 'http://nomisma.org/id/licinius'}],
 'http://nomisma.org/ontology#hasDenomination': [{'@id': 'http://nomisma.org/id/ae2'}],
 'http://nomisma.org/ontology#hasEndDate': [{'@type': 'http://www.w3.org/2001/XMLSchema#gYear',
   '@value': '0311-01-01'}],
 'http://nomisma.org/ontology#hasManufacture': [{'@id': 'http://nomisma.org/id/struck'}],
 'http://nomisma.org/ontology#hasMaterial': [{'@id': 'http://nomisma.org/id/ae'}],
 'http://nomisma.org/ontology#hasMint': [{'@id': 'http://nomisma.org/id/siscia'}],
 'http://nomisma.org/ontology#hasObverse': [{'@id': 'http://numismatics.org/ocre/id/ric.6.sis.221#obverse'}],
 'http://nomisma.org/ontology#hasRegion': [{'@id': 'http://nomisma.org/id/pannonia'}],
 'http://nomisma.org/ontology#hasReverse': [{'@id': 'http://numismatics.org/ocre/

In [33]:
len(nomisma_json)

169803

In [53]:
provenances_ids = [el["@id"] for el in nomisma_json if "#pro" in el["@id"]]
provenances_ids[:5]

['http://numismatics.org/ocre/symbol/monogram.ric.10.theodosius_ii.5#provenance',
 'http://numismatics.org/ocre/symbol/monogram.ric.10.anthemius.7#provenance',
 'http://numismatics.org/ocre/symbol/monogram.ric.10.zeno.14#provenance',
 'http://numismatics.org/ocre/symbol/monogram.ric.10.marcian.2#provenance',
 'http://numismatics.org/ocre/symbol/monogram.ric.10.zeno.12#provenance']

# Extracting coins alone

In [228]:
# filter for coins only
with_object_type = [el for el in nomisma_json if "http://nomisma.org/ontology#representsObjectType" in el.keys()]
coins_json = [el for el in with_object_type if el["http://nomisma.org/ontology#representsObjectType"][0]["@id"] ==  'http://nomisma.org/id/coin']

In [229]:
len(coins_json)

54318

In [230]:
coins_json_df = pd.DataFrame(coins_json)
coins_json_df.head(5)

,@id,@type,http://nomisma.org/ontology#hasAuthority,http://nomisma.org/ontology#hasDenomination,http://nomisma.org/ontology#hasEndDate,http://nomisma.org/ontology#hasManufacture,http://nomisma.org/ontology#hasMaterial,http://nomisma.org/ontology#hasMint,http://nomisma.org/ontology#hasObverse,http://nomisma.org/ontology#hasRegion,http://nomisma.org/ontology#hasReverse,http://nomisma.org/ontology#hasStartDate,http://nomisma.org/ontology#representsObjectType,http://purl.org/dc/terms/source,http://rdfs.org/ns/void#inDataset,http://www.w3.org/2004/02/skos/core#definition,http://www.w3.org/2004/02/skos/core#prefLabel,http://www.w3.org/2004/02/skos/core#broader,http://purl.org/dc/terms/replaces,http://purl.org/dc/terms/isReplacedBy,http://www.w3.org/2004/02/skos/core#exactMatch,http://nomisma.org/ontology#hasIssuer
0,http://numismatics.org/ocre/id/ric.6.sis.221,"[http://www.w3.org/2004/02/skos/core#Concept, ...",[{'@id': 'http://nomisma.org/id/licinius'}],[{'@id': 'http://nomisma.org/id/ae2'}],[{'@type': 'http://www.w3.org/2001/XMLSchema#g...,[{'@id': 'http://nomisma.org/id/struck'}],[{'@id': 'http://nomisma.org/id/ae'}],[{'@id': 'http://nomisma.org/id/siscia'}],[{'@id': 'http://numismatics.org/ocre/id/ric.6...,[{'@id': 'http://nomisma.org/id/pannonia'}],[{'@id': 'http://numismatics.org/ocre/id/ric.6...,[{'@type': 'http://www.w3.org/2001/XMLSchema#g...,[{'@id': 'http://nomisma.org/id/coin'}],[{'@id': 'http://nomisma.org/id/ric'}],[{'@id': 'http://numismatics.org/ocre/'}],"[{'@language': 'en', '@value': 'RIC VI Siscia ...","[{'@language': 'en', '@value': 'RIC VI Siscia ...",NaN,NaN,NaN,NaN,NaN
1,http://numismatics.org/ocre/id/ric.4.ph_i.28A,"[http://www.w3.org/2004/02/skos/core#Concept, ...",[{'@id': 'http://nomisma.org/id/philip_the_ara...,[{'@id': 'http://nomisma.org/id/aureus'}],[{'@type': 'http://www.w3.org/2001/XMLSchema#g...,[{'@id': 'http://nomisma.org/id/struck'}],[{'@id': 'http://nomisma.org/id/av'}],[{'@id': 'http://nomisma.org/id/rome'}],[{'@id': 'http://numismatics.org/ocre/id/ric.4...,[{'@id': 'http://nomisma.org/id/italy'}],[{'@id': 'http://numismatics.org/ocre/id/ric.4...,[{'@type': 'http://www.w3.org/2001/XMLSchema#g...,[{'@id': 'http://nomisma.org/id/coin'}],[{'@id': 'http://nomisma.org/id/ric'}],[{'@id': 'http://numismatics.org/ocre/'}],"[{'@language': 'en', '@value': 'RIC IV Philip ...","[{'@language': 'en', '@value': 'RIC IV Philip ...",[{'@id': 'http://numismatics.org/ocre/id/ric.4...,NaN,NaN,NaN,NaN
2,http://numismatics.org/ocre/id/ric.9.tri.96B.2,"[http://nomisma.org/ontology#TypeSeriesItem, h...",[{'@id': 'http://nomisma.org/id/valentinian_ii'}],[{'@id': 'http://nomisma.org/id/ae3'}],[{'@type': 'http://www.w3.org/2001/XMLSchema#g...,[{'@id': 'http://nomisma.org/id/struck'}],[{'@id': 'http://nomisma.org/id/ae'}],[{'@id': 'http://nomisma.org/id/treveri'}],[{'@id': 'http://numismatics.org/ocre/id/ric.9...,[{'@id': 'http://nomisma.org/id/gallia'}],[{'@id': 'http://numismatics.org/ocre/id/ric.9...,[{'@type': 'http://www.w3.org/2001/XMLSchema#g...,[{'@id': 'http://nomisma.org/id/coin'}],[{'@id': 'http://nomisma.org/id/ric'}],[{'@id': 'http://numismatics.org/ocre/'}],"[{'@language': 'en', '@value': 'RIC IX Treveri...","[{'@language': 'en', '@value': 'RIC IX Treveri...",[{'@id': 'http://numismatics.org/ocre/id/ric.9...,NaN,NaN,NaN,NaN
3,http://numismatics.org/ocre/id/ric.8.aq.3,"[http://www.w3.org/2004/02/skos/core#Concept, ...",[{'@id': 'http://nomisma.org/id/constantine_ii'}],[{'@id': 'http://nomisma.org/id/multiple-gold'}],[{'@type': 'http://www.w3.org/2001/XMLSchema#g...,[{'@id': 'http://nomisma.org/id/struck'}],[{'@id': 'http://nomisma.org/id/av'}],[{'@id': 'http://nomisma.org/id/aquileia'}],[{'@id': 'http://numismatics.org/ocre/id/ric.8...,[{'@id': 'http://nomisma.org/id/italy'}],[{'@id': 'http://numismatics.org/ocre/id/ric.8...,[{'@type': 'http://www.w3.org/2001/XMLSchema#g...,[{'@id': 'http://nomisma.org/id/coin'}],[{'@id': 'http://nomisma.org/id/ric'}],[{'@id': 'http://numismatics.org/ocre/'}],"[{'@language': 

# Extracting Obverse and Reverse

In [231]:
# filter for reverses only
coins_reverses_json = [el for el in nomisma_json if "#reverse" in el["@id"]]
len(coins_reverses_json)

51855

In [232]:
coins_reverses_df = pd.DataFrame(coins_reverses_json)
coins_reverses_df.columns = [col + "_reverse" for col in coins_reverses_df.columns]
coins_reverses_df.head(5)

,@id_reverse,http://nomisma.org/ontology#hasLegend_reverse,http://nomisma.org/ontology#hasPortrait_reverse,http://purl.org/dc/terms/description_reverse,http://nomisma.org/ontology#hasControlmark_reverse
0,http://numismatics.org/ocre/id/ric.10.leo_i_e....,[{'@value': 'VICTORIA AVGG'}],[{'@id': 'http://collection.britishmuseum.org/...,"[{'@language': 'en', '@value': 'Victory, winge...",NaN
1,http://numismatics.org/ocre/id/ric.4.ge.191B#r...,[{'@value': 'VICTORIAE BRITTANICAE S C'}],[{'@id': 'http://collection.britishmuseum.org/...,"[{'@language': 'en', '@value': 'Victory, winge...",NaN
2,http://numismatics.org/ocre/id/ric.1(2).vit.15...,[{'@value': 'VICTOR AVGVS S C'}],[{'@id': 'http://collection.britishmuseum.org/...,"[{'@language': 'en', '@value': 'Victory, drape...",NaN
3,http://numismatics.org/ocre/id/ric.4.crl.149b#...,[{'@value': 'VIRTVS AVGG'}],[{'@id': 'http://collection.britishmuseum.org/...,"[{'@language': 'en', '@value': 'Virtus, helmet...",NaN
4,http://numismatics.org/ocre/id/ric.6.carth.24a...,[{'@value': 'FELIX ADV-ENT AVGG NN or FELIX A-...,[{'@id': 'file:///home/kasev/coins_etl/data/no...,"[{'@language': 'en', '@value': 'Africa, wearin...",NaN


In [233]:
coins_reverses_df["@id_reverse"] = coins_reverses_df["@id_reverse"].apply(lambda x: x.rpartition("/")[2])

In [234]:
# filter for obverses only
coins_obverses_json = [el for el in nomisma_json if "#obverse" in el["@id"]]
len(coins_obverses_json)

51858

In [235]:
coins_obverses_df = pd.DataFrame(coins_obverses_json)
coins_obverses_df.columns = [col + "_obverse" for col in coins_obverses_df.columns]
coins_obverses_df.head(5)

,@id_obverse,http://nomisma.org/ontology#hasLegend_obverse,http://nomisma.org/ontology#hasPortrait_obverse,http://purl.org/dc/terms/description_obverse
0,http://numismatics.org/ocre/id/ric.4.el.160#ob...,[{'@value': 'IMP ANTONINVS AVG'}],[{'@id': 'http://nomisma.org/id/elagabalus'}],"[{'@language': 'en', '@value': 'Bust of Elagab..."
1,http://numismatics.org/ocre/id/ric.9.rom.24B.i...,[{'@value': 'D N VALEN-S P F AVG'}],[{'@id': 'http://nomisma.org/id/valens'}],"[{'@language': 'en', '@value': 'Bust of Valens..."
2,http://numismatics.org/ocre/id/ric.9.lug.21B.x...,[{'@value': 'D N GRATIAN-VS AVGG AVG'}],[{'@id': 'http://nomisma.org/id/gratian'}],"[{'@language': 'en', '@value': 'Bust of Gratia..."
3,http://numismatics.org/ocre/id/ric.4.pn.39#obv...,[{'@value': 'IMP CAES C PESC NIGER IVST COS II'}],[{'@id': 'http://nomisma.org/id/pescennius_nig...,"[{'@language': 'en', '@value': 'Head of Pescen..."
4,http://numismatics.org/ocre/id/ric.4.el.147#ob...,[{'@value': 'IMP ANTONINVS PIVS AVG'}],[{'@id': 'http://nomisma.org/id/elagabalus'}],"[{'@language': 'en', '@value': 'Bust of Elagab..."


In [236]:
coins_obverses_df["@id_obverse"] = coins_obverses_df["@id_obverse"].apply(lambda x: x.rpartition("/")[2])

In [237]:
# "@id"
columns_id = ["http://nomisma.org/ontology#hasAuthority", 
              "http://nomisma.org/ontology#hasDenomination",
              "http://nomisma.org/ontology#hasManufacture",
              "http://nomisma.org/ontology#hasMaterial",
              "http://nomisma.org/ontology#hasMint",
              "http://nomisma.org/ontology#hasRegion",
              "http://purl.org/dc/terms/source",
              "http://rdfs.org/ns/void#inDataset",
              "http://www.w3.org/2004/02/skos/core#broader",
              "http://purl.org/dc/terms/replaces",
              "http://purl.org/dc/terms/isReplacedBy",
              "http://www.w3.org/2004/02/skos/core#exactMatch",
              "http://nomisma.org/ontology#hasIssuer",
              "http://nomisma.org/ontology#representsObjectType",
              
              "http://nomisma.org/ontology#hasObverse",
              "http://nomisma.org/ontology#hasReverse",
             
             "http://nomisma.org/ontology#hasPortrait_obverse",
              "http://nomisma.org/ontology#hasPortrait_reverse"
             ]
 
# "@value"
columns_value = ["http://www.w3.org/2004/02/skos/core#definition",
                 "http://www.w3.org/2004/02/skos/core#prefLabel",
                 "http://nomisma.org/ontology#hasEndDate", 
                 "http://nomisma.org/ontology#hasStartDate",
                 
                "http://nomisma.org/ontology#hasLegend_obverse",
                "http://nomisma.org/ontology#hasLegend_reverse",
                "http://purl.org/dc/terms/description_obverse",
                "http://purl.org/dc/terms/description_reverse",
                ]

def extract_id(cell):
    try: return cell[0]["@id"].rpartition("/")[2]
    except: return ""

for col in columns_id:
    try: coins_json_df[col] = coins_json_df[col].apply(extract_id)
    except: pass
    try: coins_obverses_df[col] = coins_obverses_df[col].apply(extract_id)
    except: pass
    try: coins_reverses_df[col] = coins_reverses_df[col].apply(extract_id)
    except: pass
    
def extract_value(cell):
    try: return cell[0]["@value"].rpartition("/")[2]
    except: return ""

for col in columns_value:
    try: coins_json_df[col] = coins_json_df[col].apply(extract_value) 
    except: pass
    try: coins_obverses_df[col] = coins_obverses_df[col].apply(extract_value) 
    except: pass
    try: coins_reverses_df[col] = coins_reverses_df[col].apply(extract_value) 
    except: pass



In [238]:
coins_obverses_df.head(5)

,@id_obverse,http://nomisma.org/ontology#hasLegend_obverse,http://nomisma.org/ontology#hasPortrait_obverse,http://purl.org/dc/terms/description_obverse
0,ric.4.el.160#obverse,IMP ANTONINVS AVG,elagabalus,"Bust of Elagabalus, laureate, draped, right"
1,ric.9.rom.24B.ixb#obverse,D N VALEN-S P F AVG,valens,"Bust of Valens, pearl-diademed, draped and cui..."
2,ric.9.lug.21B.xiva#obverse,D N GRATIAN-VS AVGG AVG,gratian,"Bust of Gratian, pearl-diademed, draped and cu..."
3,ric.4.pn.39#obverse,IMP CAES C PESC NIGER IVST COS II,pescennius_niger,"Head of Pescennius Niger, laureate, right"
4,ric.4.el.147#obverse,IMP ANTONINVS PIVS AVG,elagabalus,"Bust of Elagabalus, horned, laureate, draped, ..."


In [239]:
coins_reverses_df.head(5)

,@id_reverse,http://nomisma.org/ontology#hasLegend_reverse,http://nomisma.org/ontology#hasPortrait_reverse,http://purl.org/dc/terms/description_reverse,http://nomisma.org/ontology#hasControlmark_reverse
0,ric.10.leo_i_e.634#reverse,VICTORIA AVGG,60915,"Victory, winged, draped, seated right on cuira...",NaN
1,ric.4.ge.191B#reverse,VICTORIAE BRITTANICAE S C,60915,"Victory, winged, draped, seated left on shield...",NaN
2,ric.1(2).vit.151#reverse,VICTOR AVGVS S C,60915,"Victory, draped, advancing left, placing shiel...",NaN
3,ric.4.crl.149b#reverse,VIRTVS AVGG,80685,"Virtus, helmeted, draped, standing left, holdi...",NaN
4,ric.6.carth.24a#reverse,FELIX ADV-ENT AVGG NN or FELIX A-D-VENT AVGG N...,nomisma.rdf,"Africa, wearing elephant-skin head-dress, drap...",NaN


In [240]:
coins_json_df.rename(columns={"http://nomisma.org/ontology#hasObverse" : "@id_obverse"}, inplace=True)
coins_json_df.rename(columns={"http://nomisma.org/ontology#hasReverse" : "@id_reverse"}, inplace=True)


coins_json_df.head(5)

,@id,@type,http://nomisma.org/ontology#hasAuthority,http://nomisma.org/ontology#hasDenomination,http://nomisma.org/ontology#hasEndDate,http://nomisma.org/ontology#hasManufacture,http://nomisma.org/ontology#hasMaterial,http://nomisma.org/ontology#hasMint,@id_obverse,http://nomisma.org/ontology#hasRegion,@id_reverse,http://nomisma.org/ontology#hasStartDate,http://nomisma.org/ontology#representsObjectType,http://purl.org/dc/terms/source,http://rdfs.org/ns/void#inDataset,http://www.w3.org/2004/02/skos/core#definition,http://www.w3.org/2004/02/skos/core#prefLabel,http://www.w3.org/2004/02/skos/core#broader,http://purl.org/dc/terms/replaces,http://purl.org/dc/terms/isReplacedBy,http://www.w3.org/2004/02/skos/core#exactMatch,http://nomisma.org/ontology#hasIssuer
0,http://numismatics.org/ocre/id/ric.6.sis.221,"[http://www.w3.org/2004/02/skos/core#Concept, ...",licinius,ae2,0311-01-01,struck,ae,siscia,ric.6.sis.221#obverse,pannonia,ric.6.sis.221#reverse,0311-01-01,coin,ric,,RIC VI Siscia 221,RIC VI Siscia 221,,,,,
1,http://numismatics.org/ocre/id/ric.4.ph_i.28A,"[http://www.w3.org/2004/02/skos/core#Concept, ...",philip_the_arab,aureus,0247-01-01,struck,av,rome,ric.4.ph_i.28A#obverse,italy,ric.4.ph_i.28A#reverse,0244-01-01,coin,ric,,RIC IV Philip I 28A,RIC IV Philip I 28A,ric.4.ph_i.28,,,,
2,http://numismatics.org/ocre/id/ric.9.tri.96B.2,"[http://nomisma.org/ontology#TypeSeriesItem, h...",valentinian_ii,ae3,0392-01-01,struck,ae,treveri,ric.9.tri.96B.2#obverse,gallia,ric.9.tri.96B.2#reverse,0388-01-01,coin,ric,,RIC IX Treveri 96B: Subtype 2,RIC IX Treveri 96B: Subtype 2,ric.9.tri.96B,,,,
3,http://numismatics.org/ocre/id/ric.8.aq.3,"[http://www.w3.org/2004/02/skos/core#Concept, ...",constantine_ii,multiple-gold,0340-01-01,struck,av,aquileia,ric.8.aq.3#obverse,italy,ric.8.aq.3#reverse,0337-01-01,coin,ric,,RIC VIII Aquileia 3,RIC VIII Aquileia 3,,,,,
4,http://numismatics.org/ocre/id/ric.4.crl.191,"[http://www.w3.org/2004/02/skos/core#Concept, ...",caracalla,denarius,0211-01-01,struck,ar,rome,ric.4.crl.191#obverse,italy,ric.4.crl.191#reverse,0211-01-01,coin,ric,,RIC IV Caracalla 191,RIC IV Caracalla 191,,,,,


In [248]:
# test:
coins_json_df.merge(coins_obverses_df, on="@id_obverse").head(5)

,@id,@type,http://nomisma.org/ontology#hasAuthority,http://nomisma.org/ontology#hasDenomination,http://nomisma.org/ontology#hasEndDate,http://nomisma.org/ontology#hasManufacture,http://nomisma.org/ontology#hasMaterial,http://nomisma.org/ontology#hasMint,@id_obverse,http://nomisma.org/ontology#hasRegion,@id_reverse,http://nomisma.org/ontology#hasStartDate,http://nomisma.org/ontology#representsObjectType,http://purl.org/dc/terms/source,http://rdfs.org/ns/void#inDataset,http://www.w3.org/2004/02/skos/core#definition,http://www.w3.org/2004/02/skos/core#prefLabel,http://www.w3.org/2004/02/skos/core#broader,http://purl.org/dc/terms/replaces,http://purl.org/dc/terms/isReplacedBy,http://www.w3.org/2004/02/skos/core#exactMatch,http://nomisma.org/ontology#hasIssuer,http://nomisma.org/ontology#hasLegend_obverse_x,http://nomisma.org/ontology#hasPortrait_obverse_x,http://purl.org/dc/terms/description_obverse_x,http://nomisma.org/ontology#hasLegend_obverse_y,http://nomisma.org/ontology#hasPortrait_obverse_y,http://purl.org/dc/terms/description_obverse_y
0,http://numismatics.org/ocre/id/ric.6.sis.221,"[http://www.w3.org/2004/02/skos/core#Concept, ...",licinius,ae2,0311-01-01,struck,ae,siscia,ric.6.sis.221#obverse,pannonia,ric.6.sis.221#reverse,0311-01-01,coin,ric,,RIC VI Siscia 221,RIC VI Siscia 221,,,,,,DIVO GAL VAL MAXIMIANO,galerius,"Head of Galerius, veiled, right",DIVO GAL VAL MAXIMIANO,galerius,"Head of Galerius, veiled, right"
1,http://numismatics.org/ocre/id/ric.4.ph_i.28A,"[http://www.w3.org/2004/02/skos/core#Concept, ...",philip_the_arab,aureus,0247-01-01,struck,av,rome,ric.4.ph_i.28A#obverse,italy,ric.4.ph_i.28A#reverse,0244-01-01,coin,ric,,RIC IV Philip I 28A,RIC IV Philip I 28A,ric.4.ph_i.28,,,,,IMP M IVL PHILIPPVS AVG,philip_the_arab,"Bust of Philip the Arab, laureate, draped, cui...",IMP M IVL PHILIPPVS AVG,philip_the_arab,"Bust of Philip the Arab, laureate, draped, cui..."
2,http://numismatics.org/ocre/id/ric.9.tri.96B.2,"[http://nomisma.org/ontology#TypeSeriesItem, h...",valentinian_ii,ae3,0392-01-01,struck,ae,treveri,ric.9.tri.96B.2#obverse,gallia,ric.9.tri.96B.2#reverse,0388-01-01,coin,ric,,RIC IX Treveri 96B: Subtype 2,RIC IX Treveri 96B: Subtype 2,ric.9.tri.96B,,,,,D N THEODO-SIVS P F AVG,theodosius_i,"Bust of Theodosius I, pearl-diademed, draped a...",D N THEODO-SIVS P F AVG,theodosius_i,"Bust of Theodosius I, pearl-diademed, draped a..."
3,http://numismatics.org/ocre/id/ric.8.aq.3,"[http://www.w3.org/2004/02/skos/core#Concept, ...",constantine_ii,multiple-gold,0340-01-01,struck,av,aquileia,ric.8.aq.3#obverse,italy,ric.8.aq.3#reverse,0337-01-01,coin,ric,,RIC VIII Aquileia 3,RIC VIII Aquileia 3,,,,,,FL IVL CONS-TANS P F AVG,constans,"Bust of Constans, laureate, rosette-diademed, ...",FL IVL CONS-TANS P F AVG,constans,"Bust of Constans, laureate, rosette-diademed, ..."
4,http://numismatics.org/ocre/id/ric.4.crl.191,"[http://www.w3.org/2004/02/skos/core#Concept, ...",caracalla,denarius,0211-01-01,struck,ar,rome,ric.4.crl.191#obverse,italy,ric.4.crl.191#reverse,0211-01-01,coin,ric,,RIC IV Caracalla 191,RIC IV Caracalla 191,,,,,,ANTONINVS PIVS AVG BRIT,caracalla,"Head of Caracalla, laureate, right",ANTONINVS PIVS AVG BRIT,caracalla,"Head of Caracalla, laureate, right"


In [245]:
coins_json_df = coins_json_df.merge(coins_obverses_df, on="@id_obverse")

In [246]:
# test
coins_json_df.merge(coins_reverses_df, on="@id_reverse").head(5)

,@id,@type,http://nomisma.org/ontology#hasAuthority,http://nomisma.org/ontology#hasDenomination,http://nomisma.org/ontology#hasEndDate,http://nomisma.org/ontology#hasManufacture,http://nomisma.org/ontology#hasMaterial,http://nomisma.org/ontology#hasMint,@id_obverse,http://nomisma.org/ontology#hasRegion,@id_reverse,http://nomisma.org/ontology#hasStartDate,http://nomisma.org/ontology#representsObjectType,http://purl.org/dc/terms/source,http://rdfs.org/ns/void#inDataset,http://www.w3.org/2004/02/skos/core#definition,http://www.w3.org/2004/02/skos/core#prefLabel,http://www.w3.org/2004/02/skos/core#broader,http://purl.org/dc/terms/replaces,http://purl.org/dc/terms/isReplacedBy,http://www.w3.org/2004/02/skos/core#exactMatch,http://nomisma.org/ontology#hasIssuer,http://nomisma.org/ontology#hasLegend_obverse,http://nomisma.org/ontology#hasPortrait_obverse,http://purl.org/dc/terms/description_obverse,http://nomisma.org/ontology#hasLegend_reverse,http://nomisma.org/ontology#hasPortrait_reverse,http://purl.org/dc/terms/description_reverse,http://nomisma.org/ontology#hasControlmark_reverse
0,http://numismatics.org/ocre/id/ric.6.sis.221,"[http://www.w3.org/2004/02/skos/core#Concept, ...",licinius,ae2,0311-01-01,struck,ae,siscia,ric.6.sis.221#obverse,pannonia,ric.6.sis.221#reverse,0311-01-01,coin,ric,,RIC VI Siscia 221,RIC VI Siscia 221,,,,,,DIVO GAL VAL MAXIMIANO,galerius,"Head of Galerius, veiled, right",FORTI FOR-TVNAE,58260,"Fortuna, draped, standing left by wheel, holdi...",NaN
1,http://numismatics.org/ocre/id/ric.4.ph_i.28A,"[http://www.w3.org/2004/02/skos/core#Concept, ...",philip_the_arab,aureus,0247-01-01,struck,av,rome,ric.4.ph_i.28A#obverse,italy,ric.4.ph_i.28A#reverse,0244-01-01,coin,ric,,RIC IV Philip I 28A,RIC IV Philip I 28A,ric.4.ph_i.28,,,,,IMP M IVL PHILIPPVS AVG,philip_the_arab,"Bust of Philip the Arab, laureate, draped, cui...",ANNONA AVGG,nomisma.rdf,"Annona, draped, standing left, holding corn-ea...",NaN
2,http://numismatics.org/ocre/id/ric.9.tri.96B.2,"[http://nomisma.org/ontology#TypeSeriesItem, h...",valentinian_ii,ae3,0392-01-01,struck,ae,treveri,ric.9.tri.96B.2#obverse,gallia,ric.9.tri.96B.2#reverse,0388-01-01,coin,ric,,RIC IX Treveri 96B: Subtype 2,RIC IX Treveri 96B: Subtype 2,ric.9.tri.96B,,,,,D N THEODO-SIVS P F AVG,theodosius_i,"Bust of Theodosius I, pearl-diademed, draped a...",GLORIA RO-MANORVM,,"Emperor advancing right, dragging captive with...",NaN
3,http://numismatics.org/ocre/id/ric.8.aq.3,"[http://www.w3.org/2004/02/skos/core#Concept, ...",constantine_ii,multiple-gold,0340-01-01,struck,av,aquileia,ric.8.aq.3#obverse,italy,ric.8.aq.3#reverse,0337-01-01,coin,ric,,RIC VIII Aquileia 3,RIC VIII Aquileia 3,,,,,,FL IVL CONS-TANS P F AVG,constans,"Bust of Constans, laureate, rosette-diademed, ...",VIRTVS - CONSTANTIS AVG,constans,"Constans, draped, cuirassed, standing front, h...",NaN
4,http://numismatics.org/ocre/id/ric.4.crl.191,"[http://www.w3.org/2004/02/skos/core#Concept, ...",caracalla,denarius,0211-01-01,struck,ar,rome,ric.4.crl.191#obverse,italy,ric.4.crl.191#reverse,0211-01-01,coin,ric,,RIC IV Caracalla 191,RIC IV Caracalla 191,,,,,,ANTONINVS PIVS AVG BRIT,caracalla,"Head of Caracalla, laureate, right",PONTIF TR P XIIII COS III,80685,"Virtus, helmeted, draped, standing right, left...",NaN


In [249]:
coins_json_df = coins_json_df.merge(coins_reverses_df, on="@id_reverse")

In [250]:
shorter_col_names = [col.rpartition("/")[2] for col in coins_json_df.columns]
shorter_col_names

['@id',
 '@type',
 'ontology#hasAuthority',
 'ontology#hasDenomination',
 'ontology#hasEndDate',
 'ontology#hasManufacture',
 'ontology#hasMaterial',
 'ontology#hasMint',
 '@id_obverse',
 'ontology#hasRegion',
 '@id_reverse',
 'ontology#hasStartDate',
 'ontology#representsObjectType',
 'source',
 'void#inDataset',
 'core#definition',
 'core#prefLabel',
 'core#broader',
 'replaces',
 'isReplacedBy',
 'core#exactMatch',
 'ontology#hasIssuer',
 'ontology#hasLegend_obverse',
 'ontology#hasPortrait_obverse',
 'description_obverse',
 'ontology#hasLegend_reverse',
 'ontology#hasPortrait_reverse',
 'description_reverse',
 'ontology#hasControlmark_reverse']

In [251]:
coins_json_df.columns = shorter_col_names

In [252]:
coins_json_df.head(5)

,@id,@type,ontology#hasAuthority,ontology#hasDenomination,ontology#hasEndDate,ontology#hasManufacture,ontology#hasMaterial,ontology#hasMint,@id_obverse,ontology#hasRegion,@id_reverse,ontology#hasStartDate,ontology#representsObjectType,source,void#inDataset,core#definition,core#prefLabel,core#broader,replaces,isReplacedBy,core#exactMatch,ontology#hasIssuer,ontology#hasLegend_obverse,ontology#hasPortrait_obverse,description_obverse,ontology#hasLegend_reverse,ontology#hasPortrait_reverse,description_reverse,ontology#hasControlmark_reverse
0,http://numismatics.org/ocre/id/ric.6.sis.221,"[http://www.w3.org/2004/02/skos/core#Concept, ...",licinius,ae2,0311-01-01,struck,ae,siscia,ric.6.sis.221#obverse,pannonia,ric.6.sis.221#reverse,0311-01-01,coin,ric,,RIC VI Siscia 221,RIC VI Siscia 221,,,,,,DIVO GAL VAL MAXIMIANO,galerius,"Head of Galerius, veiled, right",FORTI FOR-TVNAE,58260,"Fortuna, draped, standing left by wheel, holdi...",NaN
1,http://numismatics.org/ocre/id/ric.4.ph_i.28A,"[http://www.w3.org/2004/02/skos/core#Concept, ...",philip_the_arab,aureus,0247-01-01,struck,av,rome,ric.4.ph_i.28A#obverse,italy,ric.4.ph_i.28A#reverse,0244-01-01,coin,ric,,RIC IV Philip I 28A,RIC IV Philip I 28A,ric.4.ph_i.28,,,,,IMP M IVL PHILIPPVS AVG,philip_the_arab,"Bust of Philip the Arab, laureate, draped, cui...",ANNONA AVGG,nomisma.rdf,"Annona, draped, standing left, holding corn-ea...",NaN
2,http://numismatics.org/ocre/id/ric.9.tri.96B.2,"[http://nomisma.org/ontology#TypeSeriesItem, h...",valentinian_ii,ae3,0392-01-01,struck,ae,treveri,ric.9.tri.96B.2#obverse,gallia,ric.9.tri.96B.2#reverse,0388-01-01,coin,ric,,RIC IX Treveri 96B: Subtype 2,RIC IX Treveri 96B: Subtype 2,ric.9.tri.96B,,,,,D N THEODO-SIVS P F AVG,theodosius_i,"Bust of Theodosius I, pearl-diademed, draped a...",GLORIA RO-MANORVM,,"Emperor advancing right, dragging captive with...",NaN
3,http://numismatics.org/ocre/id/ric.8.aq.3,"[http://www.w3.org/2004/02/skos/core#Concept, ...",constantine_ii,multiple-gold,0340-01-01,struck,av,aquileia,ric.8.aq.3#obverse,italy,ric.8.aq.3#reverse,0337-01-01,coin,ric,,RIC VIII Aquileia 3,RIC VIII Aquileia 3,,,,,,FL IVL CONS-TANS P F AVG,constans,"Bust of Constans, laureate, rosette-diademed, ...",VIRTVS - CONSTANTIS AVG,constans,"Constans, draped, cuirassed, standing front, h...",NaN
4,http://numismatics.org/ocre/id/ric.4.crl.191,"[http://www.w3.org/2004/02/skos/core#Concept, ...",caracalla,denarius,0211-01-01,struck,ar,rome,ric.4.crl.191#obverse,italy,ric.4.crl.191#reverse,0211-01-01,coin,ric,,RIC IV Caracalla 191,RIC IV Caracalla 191,,,,,,ANTONINVS PIVS AVG BRIT,caracalla,"Head of Caracalla, laureate, right",PONTIF TR P XIIII COS III,80685,"Virtus, helmeted, draped, standing right, left...",NaN


In [253]:
len(coins_json_df)

51311

In [ ]:
coins_json_df.to_json("../data/coins_raw_df.json")

In [8]:
s = sddk.cloudSession("sciencedata.dk", "SDAM_root", "648597@au.dk")

connection with shared folder established with you as its owner
endpoint variable has been configured to: https://sciencedata.dk/files/SDAM_root/


In [257]:
s.write_file("SDAM_data/coins/coins_df.json", coins_json_df)

Your <class 'pandas.core.frame.DataFrame'> object has been succefully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/coins/coins_df.json"


In [261]:
# to access gsheet, you need Google Service Account key json file
# I have mine located in my personal space on sciencedata.dk, so I read it from there:

# (1) read the file and parse its content
file_data = sddk.read_file("https://sciencedata.dk/files/ServiceAccountsKey.json", "dict", conf)# conf[0].get(conf[1] + "ServiceAccountsKey.json").json()
# (2) transform the content into crendentials object
credentials = service_account.Credentials.from_service_account_info(file_data)
# (3) specify your usage of the credentials
scoped_credentials = credentials.with_scopes(['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive'])
# (4) use the constrained credentials for authentication of gspread package
gc = gspread.Client(auth=scoped_credentials)
# (5) establish connection with spreadsheets specified by their url
coins_gsheet = gc.open_by_url("https://docs.google.com/spreadsheets/d/1T5krnx1fzu_MrJTmfu3hS1Jpx_PC79D2J7cnkoJFi4s/edit?usp=sharing")

In [262]:
set_with_dataframe(coins_gsheet.add_worksheet("coins_df_complete", 1,1), coins_json_df)